In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import pytz
import glob
import math
import time
from dateutil.parser import isoparse
from datetime import datetime, timedelta
from DirectoryGenerator import DirectoryGenerator

In [ ]:
dirGen = DirectoryGenerator()

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
totalDays = abs(endDate - startDate).days + 1
totalDays

In [ ]:
def parseTimestamp(timestamp):
    try:
        return isoparse(timestamp).astimezone(pytz.utc)
    except:
        return np.nan

In [ ]:
discussionForumsDataPostsFilesPath = str(input())

In [ ]:
discussionForumsDataPostsFiles = glob.glob(discussionForumsDataPostsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
discussionForumsPostsData = pd.DataFrame()
for discussionForumsDataPostsFile in discussionForumsDataPostsFiles:
    discussionForumsPostsData = discussionForumsPostsData.append(pd.read_csv(discussionForumsDataPostsFile)[['metadata_event_time', 'metadata_context_id', 'metadata_context_role', 'metadata_user_id', 'body_assignment_id', 'body_discussion_topic_id', 'body_parent_discussion_entry_id']], ignore_index = True)
    
discussionForumsPostsData

In [ ]:
discussionForumsPostsData['metadata_event_time'] = discussionForumsPostsData['metadata_event_time'].apply(parseTimestamp)
discussionForumsPostsData.dropna(subset=['metadata_event_time'], inplace = True)
discussionForumsPostsData

In [ ]:
discussionForumsPostsData = discussionForumsPostsData.loc[(discussionForumsPostsData['metadata_event_time'] >= startDate) & (discussionForumsPostsData['metadata_event_time'] <= endDate)]
discussionForumsPostsData

In [ ]:
discussionForumsPostsData = discussionForumsPostsData.loc[discussionForumsPostsData['metadata_context_role'] == 'StudentEnrollment']
discussionForumsPostsData.drop(['metadata_context_role'], axis = 1, inplace = True)
discussionForumsPostsData

In [ ]:
discussionForumsPostsData.drop(['body_parent_discussion_entry_id'], axis = 1, inplace = True)
discussionForumsPostsData

In [ ]:
courseTopicGradedUngradedList = pd.read_csv(str(input()))
courseTopicGradedUngradedList

In [ ]:
discussionForumsPostsData['course_topic'] = discussionForumsPostsData['metadata_context_id'] + "-" + discussionForumsPostsData['body_discussion_topic_id']
discussionForumsPostsData

In [ ]:
discussionForumsPostsData = discussionForumsPostsData.merge(courseTopicGradedUngradedList, on=['course_topic'])
discussionForumsPostsData

In [ ]:
discussionForumsPostsDataGraded = discussionForumsPostsData.loc[discussionForumsPostsData['is_graded'] == True]
discussionForumsPostsDataGraded.drop(['is_graded'], axis = 1, inplace = True)
discussionForumsPostsDataGraded

In [ ]:
discussionForumsPostsDataGraded = discussionForumsPostsDataGraded.sort_values(by=['metadata_event_time']).drop_duplicates(subset=['metadata_context_id', 'body_discussion_topic_id', 'metadata_user_id'])
discussionForumsPostsDataGraded

In [ ]:
assignmentsCreationInfoFilesPath = str(input())

In [ ]:
assignmentsCreationInfoFiles = glob.glob(assignmentsCreationInfoFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
assignmentsCreationInfoData = pd.DataFrame()
for assignmentsCreationInfoFile in assignmentsCreationInfoFiles:
    assignmentsCreationInfoData = assignmentsCreationInfoData.append(pd.read_csv(assignmentsCreationInfoFile), ignore_index = True)
    
assignmentsCreationInfoData

In [ ]:
assignmentsCreationInfoData = assignmentsCreationInfoData[['metadata_event_time', 'body_context_id', 'body_discussion_topic_id']]
assignmentsCreationInfoData

In [ ]:
assignmentsCreationInfoData['metadata_event_time'] = assignmentsCreationInfoData['metadata_event_time'].apply(isoparse)
assignmentsCreationInfoData.dropna(subset=['metadata_event_time'], inplace = True)
assignmentsCreationInfoData = assignmentsCreationInfoData.loc[assignmentsCreationInfoData['metadata_event_time'] <= endDate]
assignmentsCreationInfoData = assignmentsCreationInfoData.sort_values(by='metadata_event_time').drop_duplicates(subset=['body_context_id', 'body_discussion_topic_id']).reset_index()
assignmentsCreationInfoData

In [ ]:
assignmentsCreationInfoData['metadata_event_time'] = assignmentsCreationInfoData['metadata_event_time'].apply(lambda ts: ts if ts >= startDate else startDate)
assignmentsCreationInfoData

In [ ]:
assignmentsCreationInfoData = assignmentsCreationInfoData.rename(columns = {'body_context_id': 'metadata_context_id'})
assignmentsCreationInfoData['body_discussion_topic_id'] = assignmentsCreationInfoData['body_discussion_topic_id'].apply(lambda x: 'topic_' + str(int(x.split("_")[-1]) - 165820000000000000))
assignmentsCreationInfoData

In [ ]:
courseDiscussionForums = assignmentsCreationInfoData.groupby(['metadata_context_id', 'body_discussion_topic_id'])['metadata_event_time'].count().to_frame('total').reset_index()
courseDiscussionForums

In [ ]:
courseDiscussionForums['course_topic'] = courseDiscussionForums['metadata_context_id'] + "-" + courseDiscussionForums['body_discussion_topic_id']
courseDiscussionForums

In [ ]:
courseDiscussionForums = courseDiscussionForums.merge(courseTopicGradedUngradedList, on=['course_topic'])
courseDiscussionForums.drop(['total', 'course_topic'], axis = 1, inplace = True)
courseDiscussionForums

In [ ]:
courseStudentsFilesPath = str(input())

In [ ]:
courseStudentsFiles = glob.glob(courseStudentsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
courseStudentsData = pd.DataFrame()
for courseStudentsFile in courseStudentsFiles:
    courseStudentsData = courseStudentsData.append(pd.read_csv(courseStudentsFile), ignore_index = True)
    
courseStudentsData

In [ ]:
courseStudentsData['metadata_event_time'] = courseStudentsData['metadata_event_time'].apply(parseTimestamp)
courseStudentsData.dropna(subset=['metadata_event_time'], inplace = True)
courseStudentsData = courseStudentsData.loc[(courseStudentsData['metadata_event_time'] >= startDate) & (courseStudentsData['metadata_event_time'] <= endDate)]
courseStudentsData

In [ ]:
courseStudentsData = courseStudentsData.groupby(['metadata_context_id', 'metadata_user_id'])['metadata_event_time'].count().to_frame('total').reset_index()
courseStudentsData.drop(['total'], axis = 1, inplace = True)
courseStudentsData

In [ ]:
courseStudentsData = courseStudentsData.loc[courseStudentsData['metadata_context_id'].isin(courseDiscussionForums['metadata_context_id'].unique())]
courseStudentsData

In [ ]:
courseStudentAssignmentsComplete = pd.DataFrame()

for course in courseDiscussionForums.loc[courseDiscussionForums['is_graded'] == True]['metadata_context_id'].unique():
    courseStudentAssignmentsCompleteCourse = pd.DataFrame({'body_discussion_topic_id': courseDiscussionForums.loc[(courseDiscussionForums['is_graded'] == True) & (courseDiscussionForums['metadata_context_id'] == course)]['body_discussion_topic_id'].unique()}).merge(pd.DataFrame({'metadata_user_id': courseStudentsData.loc[courseStudentsData['metadata_context_id'] == course]['metadata_user_id'].unique()}), how = 'cross')
    courseStudentAssignmentsCompleteCourse['metadata_context_id'] = course
    courseStudentAssignmentsComplete = courseStudentAssignmentsComplete.append(courseStudentAssignmentsCompleteCourse, ignore_index = True)

courseStudentAssignmentsComplete

In [ ]:
discussionForumsPostsDataGradedFirstPost = discussionForumsPostsDataGraded.groupby(['metadata_context_id', 'body_discussion_topic_id'])['metadata_event_time'].min().to_frame('first_post').reset_index()
discussionForumsPostsDataGradedFirstPost

In [ ]:
discussionForumsPostsDataGraded = discussionForumsPostsDataGraded.merge(discussionForumsPostsDataGradedFirstPost, on=['metadata_context_id', 'body_discussion_topic_id'])
discussionForumsPostsDataGraded

In [ ]:
def calculateDelay(df):
    return (time.mktime(df['metadata_event_time'].timetuple()) / 3600) - (time.mktime(df['first_post'].timetuple()) / 3600)

discussionForumsPostsDataGraded['delay_assignments'] = discussionForumsPostsDataGraded.apply(calculateDelay, axis = 1)
discussionForumsPostsDataGraded.drop(['metadata_event_time', 'first_post'], axis = 1, inplace = True)
discussionForumsPostsDataGraded

In [ ]:
discussionForumsPostsDataGraded = discussionForumsPostsDataGraded[['metadata_context_id', 'metadata_user_id', 'body_discussion_topic_id', 'delay_assignments']]
discussionForumsPostsDataGraded

In [ ]:
discussionForumsPostsDataGraded = courseStudentAssignmentsComplete.merge(discussionForumsPostsDataGraded, how='left', on=['metadata_context_id', 'metadata_user_id', 'body_discussion_topic_id'])
discussionForumsPostsDataGraded

In [ ]:
discussionForumsPostsDataGraded = discussionForumsPostsDataGraded.fillna(((time.mktime((endDate + timedelta(seconds = 1)).timetuple()) / 3600) - (time.mktime(startDate.timetuple()) / 3600)))
discussionForumsPostsDataGraded

In [ ]:
discussionForumsPostsDataGraded = discussionForumsPostsDataGraded.groupby(['metadata_context_id', 'metadata_user_id'])['delay_assignments'].mean().to_frame('ave_delay').reset_index()
discussionForumsPostsDataGraded

In [ ]:
discussionForumsPostsDataGraded.rename(columns={"ave_delay": "ave_delay_GD"}).to_csv("GradedDiscussionForumsDelays_Intercession_2022.csv", header = True, index = False)

In [ ]:
discussionForumsPostsDataUngraded = discussionForumsPostsData.loc[discussionForumsPostsData['is_graded'] == False]
discussionForumsPostsDataUngraded.drop(['is_graded'], axis = 1, inplace = True)
discussionForumsPostsDataUngraded

In [ ]:
courseStudentAssignmentsComplete = pd.DataFrame()

for course in courseDiscussionForums.loc[courseDiscussionForums['is_graded'] == False]['metadata_context_id'].unique():
    courseStudentAssignmentsCompleteCourse = pd.DataFrame({'body_discussion_topic_id': courseDiscussionForums.loc[(courseDiscussionForums['is_graded'] == False) & (courseDiscussionForums['metadata_context_id'] == course)]['body_discussion_topic_id'].unique()}).merge(pd.DataFrame({'metadata_user_id': courseStudentsData.loc[courseStudentsData['metadata_context_id'] == course]['metadata_user_id'].unique()}), how = 'cross')
    courseStudentAssignmentsCompleteCourse['metadata_context_id'] = course
    courseStudentAssignmentsComplete = courseStudentAssignmentsComplete.append(courseStudentAssignmentsCompleteCourse, ignore_index = True)

courseStudentAssignmentsComplete

In [ ]:
discussionForumsPostsDataUngradedFirstPost = discussionForumsPostsDataUngraded.groupby(['metadata_context_id', 'body_discussion_topic_id'])['metadata_event_time'].min().to_frame('first_post').reset_index()
discussionForumsPostsDataUngradedFirstPost

In [ ]:
discussionForumsPostsDataUngraded = discussionForumsPostsDataUngraded.merge(discussionForumsPostsDataUngradedFirstPost, on=['metadata_context_id', 'body_discussion_topic_id'])
discussionForumsPostsDataUngraded

In [ ]:
discussionForumsPostsDataUngraded['delay_assignments'] = discussionForumsPostsDataUngraded.apply(calculateDelay, axis = 1)
discussionForumsPostsDataUngraded.drop(['metadata_event_time', 'first_post'], axis = 1, inplace = True)
discussionForumsPostsDataUngraded

In [ ]:
discussionForumsPostsDataUngraded = discussionForumsPostsDataUngraded[['metadata_context_id', 'metadata_user_id', 'body_discussion_topic_id', 'delay_assignments']]
discussionForumsPostsDataUngraded

In [ ]:
discussionForumsPostsDataUngraded = courseStudentAssignmentsComplete.merge(discussionForumsPostsDataUngraded, how='left', on=['metadata_context_id', 'metadata_user_id', 'body_discussion_topic_id'])
discussionForumsPostsDataUngraded

In [ ]:
discussionForumsPostsDataUngraded = discussionForumsPostsDataUngraded.fillna(((time.mktime((endDate + timedelta(seconds = 1)).timetuple()) / 3600) - (time.mktime(startDate.timetuple()) / 3600)))
discussionForumsPostsDataUngraded

In [ ]:
#discussionForumsPostsDataUngraded['delay_assignments'] = discussionForumsPostsDataUngraded.apply(setDelayToEndOfSemester, axis = 1)
#discussionForumsPostsDataUngraded

In [ ]:
#discussionForumsPostsDataUngraded.drop(['creation_time'], axis = 1, inplace = True)
#discussionForumsPostsDataUngraded

In [ ]:
discussionForumsPostsDataUngraded = discussionForumsPostsDataUngraded.groupby(['metadata_context_id', 'metadata_user_id'])['delay_assignments'].mean().to_frame('ave_delay').reset_index()
discussionForumsPostsDataUngraded

In [ ]:
discussionForumsPostsDataUngraded.rename(columns={"ave_delay": "ave_delay_UD"}).to_csv("UngradedDiscussionForumsDelays_Intercession_2022.csv", header = True, index = False)